In [1]:
import networkx as nx
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from flinspect.parse_tree import ParseTree
from flinspect.parse_forest import ParseForest

In [4]:
build_dirs = [
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/",
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6"
]

parse_tree_paths = [f for dir in build_dirs for f in Path(dir).glob("*_ptree")]

In [5]:
pf = ParseForest(parse_tree_paths)

In [6]:
nr = pf.registry

# Module Dependency Analysis

In [7]:
G = pf.get_module_dependency_graph()

Skipped 8 modules with unknown parse tree paths: ['netcdf', 'iso_c_binding', 'iso_fortran_env', 'mpi', 'netcdf_nf_data', 'netcdf_nf_interfaces', 'netcdf4_nf_interfaces', 'omp_lib']


In [8]:
# Determine source paths via the path_names files in build dirs
# This will be used to color nodes and edges based on where modules are located
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [9]:
# Find all nodes where /FMS/ is in the source path
fms_nodes = [node for node in G.nodes() if "/FMS/" in path_names.get(G.nodes[node].get('source_name','').lower(),'')]

In [10]:
# Find all fms_nodes that have no incoming edges (i.e., no dependencies)
fms_root_nodes = [node for node in fms_nodes if G.in_degree(node) == 0]

for fms_root_node in fms_root_nodes:
    print(fms_root_node, path_names.get(G.nodes[fms_root_node].get('source_name','').lower(),''))

grid_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mosaic/grid.F90
fms_yaml_output_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/parser/fms_yaml_output.F90
fm_yaml_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/field_manager/fm_yaml.F90
fms /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/libFMS.F90


In [11]:
# Find all FMS nodes that is not needed by any other FMS nodes
fms_leaf_nodes = [node for node in fms_nodes if G.out_degree(node) == 0]

for fms_leaf_node in fms_leaf_nodes:
    print(fms_leaf_node, path_names.get(G.nodes[fms_leaf_node].get('source_name','').lower(),''))

platform_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/platform/platform.F90
mersennetwister_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/random_numbers/mersennetwister.F90
fms_yaml_output_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/parser/fms_yaml_output.F90
fm_yaml_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/field_manager/fm_yaml.F90
yaml_parser_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/parser/yaml_parser.F90


In [12]:
# Find all MOM6 nodes:
mom6_nodes = [node for node in G.nodes() if "/MOM6/" in path_names.get(G.nodes[node].get('source_name','').lower(),'')]

# Find all FMS nodes that is not needed by any other MOM6 nodes recursively (i.e. transitive closure)
mom6_deps = set()
for mom6_node in mom6_nodes:
    mom6_deps.update(nx.descendants(G, mom6_node))

# Find all FMS nodes that are not needed by any other MOM6 nodes
fms_leaf_nodes = [node for node in fms_nodes if all(pred not in mom6_deps for pred in G.predecessors(node))]

for fms_leaf_node in fms_leaf_nodes:
    print(fms_leaf_node, path_names.get(G.nodes[fms_leaf_node].get('source_name','').lower(),''))

grid_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mosaic/grid.F90
mosaic_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mosaic/mosaic.F90
fms_yaml_output_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/parser/fms_yaml_output.F90
fm_yaml_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/field_manager/fm_yaml.F90
fms /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/libFMS.F90
gradient_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mosaic/gradient.F90


# Call Graph Analysis

In [13]:
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [14]:
gsc = pf.get_call_graph()

Total unfound calls across all parse trees: 178
Total unfound function calls across all parse trees: 74617


In [15]:
# Find all FMS subroutines and MOM6 subroutines
all_fms_routines = []
all_mom6_routines = []
for node in gsc.nodes():
    try:
        module_path = node.program_unit.parse_tree_path
        source_path = path_names.get(module_path.stem.lower(),'')
    except AttributeError:
        continue
    if "/MOM6/" in source_path:
        all_mom6_routines.append(node)
    elif "/FMS/" in source_path:
        all_fms_routines.append(node)

In [16]:
len(all_mom6_routines), len(all_fms_routines)

(3525, 2501)

In [17]:
def get_fms_subroutines_needed_by_mom6(gsc, all_mom6_routines):
    fms_subroutines_needed_by_mom6 = set()
    for mom6_routine in all_mom6_routines:
        descendants = nx.descendants(gsc, mom6_routine)
        for desc in descendants:
            try:
                module_path = desc.program_unit.parse_tree_path
                source_path = path_names.get(module_path.stem.lower(),'')
            except AttributeError:
                continue
            if "/FMS/" in source_path:
                fms_subroutines_needed_by_mom6.add(desc)
    return fms_subroutines_needed_by_mom6

In [18]:
fms_subroutines_needed_by_mom6 = get_fms_subroutines_needed_by_mom6(gsc, all_mom6_routines)

In [19]:
len(fms_subroutines_needed_by_mom6)

398

In [20]:
fms_subroutines_NOT_needed_by_mom6 = [r for r in all_fms_routines if r not in fms_subroutines_needed_by_mom6]
len(fms_subroutines_NOT_needed_by_mom6)

2103

In [21]:
fms_subroutines_NOT_needed_by_mom6[0]

Subroutine('field_manager_init')